# Analysis of students performance

## Set up block
*==> It must be executed minimum one time before executing next code blocks*

In [1]:
# === All imports ===
## global imports
import os
import sys
## spark imports
from pyspark.sql import SparkSession

# === Get data souce file ===
data_file = "./data/StudentsPerformance.csv"
if os.path.exists(data_file):
    print(f"[ok]: data source file is found: {data_file}")
else:
    print(f"[ko]: data source file is missing: {data_file}\n/!\The following code blocks will not be able to function normally until this problem is resolved")
    sys.exit(1)

# === Create the Spark session
try:
    spark = SparkSession.builder.master("local").appName("# Analysis of students performance").getOrCreate()
    print("[ok]: spark session created")
except Exception as e:
    print(f"[ko]: cannot create spark session: {e}\n/!\The following code blocks will not be able to function normally until this problem is resolved")
    sys.exit(1)

[ok]: data source file is found: ./data/StudentsPerformance.csv
[ok]: spark session created
